In [11]:
# Dependencies
import requests
import pandas as pd
import matplotlib as plt
import numpy as np
import json
import sqlalchemy
from sqlalchemy import create_engine
from pymongo import MongoClient

In [13]:
# Save url as variable
url = "https://us-states.p.rapidapi.com/all"

# Set headers
headers = {
	"X-RapidAPI-Key": "575ca32490msh5a85e1221a9daf0p10835bjsnfdf17e56a208",
	"X-RapidAPI-Host": "us-states.p.rapidapi.com"
}

# Make a GET request and save in variable
response = requests.request("GET", url, headers=headers)
data = response.json()

# Print the response
#print(response.text).head()

In [14]:
# Write the data to a json file
with open('data.json', 'w') as outfile:
    json.dump(data, outfile, indent=4)

In [15]:
# Create a pandas data frame
df = pd.json_normalize(response.json())
df.head()

,name,postal,date,nickname,fips,demonym,cities,lowest_point,highest_point,website,per_capita_income,median_household_income,status,other_nicknames,standard_federal_region,ap_abbreviation,gpo_abbreviation,representatives,time_zones,koppen_climate,motto,subdivisions,capital.name,capital.latitude,capital.longitude,population.density_km,population.total,population.density_mi,elevation.min_ft,elevation.min_m,elevation.mean_ft,elevation.max_rank,elevation.max_ft,elevation.span_ft,elevation.mean_rank,elevation.span_m,elevation.mean_m,elevation.max_m,area.total_mi,area.land_rank,area.land_km,area.water_rank,area.total_rank,area.land_mi,area.water_km,area.total_km,area.land_percent,area.water_mi,area.water_percent,census_bureau.region,census_bureau.division,symbols.mascot,symbols.fruit,symbols.tree fruit,symbols.soil,symbols.game bird,symbols.reptile,symbols.amphibian,symbols.nut,symbols.vegetable,symbols.gem,symbols.rock,symbols.saltwater fish,symbols.mammal,symbols.butterfly,symbols.cake,symbols.bird,symbols.insect,symbols.mineral,symbols.song,symbols.spirit,symbols.tree,symbols.fossil,symbols.freshwater fish,symbols.flower,symbols.horse,symbols.wildflower,symbols.shell,symbols.land mammal,symbols.fish,symbols.bolt action rifle,symbols.dog,symbols.marine mammal,symbols.sport,symbols.plant,symbols.metal,symbols.colors,symbols.drink,symbols.firearm,symbols.dinosaur,symbols.neckwear,symbols.floral emblem,symbols.folk dance,symbols.grape,symbols.fruit and vegetable,symbols.musical instrument,symbols.primitive fish,symbols.grain,symbols.cooking pot,symbols.lichen,symbols.grass,symbols.marine reptile,symbols.dance,symbols.tartan,symbols.gold rush ghost town,symbols.silver rush ghost town,symbols.marine fish,symbols.fabric,symbols.pets,symbols.cactus,symbols.animal,symbols.hero,symbols.shellfish,symbols.children's flower,symbols.heroine,symbols.polka,symbols.marine animal,symbols.macroinvertebrate,symbols.wildlife animal,symbols.dessert,symbols.bug,symbols.beverage,symbols.herb,symbols.saltwater mammal,symbols.heritage cattle breed,symbols.pie,symbols.stone,symbols.tortoise,symbols.saltwater reptile,symbols.prepared food,symbols.waltz,symbols.cold water gamefish,symbols.crop,symbols.raptor,symbols.exercise,symbols.artifact,symbols.snackfood,symbols.prairie grass,symbols.slogan,symbols.river,symbols.rifle,symbols.red wine grape,symbols.white wine grape,symbols.flying fossil,symbols.gun,symbols.music,symbols.wild animal game species,symbols.sports car,symbols.crustacean,symbols.meat pie,symbols.berry,symbols.treat,symbols.ballad,symbols.cat,symbols.team sport,symbols.donut,symbols.bean,symbols.cookie,symbols.muffin,symbols.folk song,symbols.game mammal,symbols.bee,symbols.mushroom,symbols.waterfowl,symbols.toy,symbols.tree nut,symbols.fruit tree,symbols.hockey team,symbols.football team,symbols.endangered species,symbols.aquatic animal,symbols.soft drink,symbols.poultry,symbols.saltwater game fish,symbols.freshwater game fish,symbols.spider,symbols.cryptid,symbols.bush,symbols.freshwater trout,symbols.salamander,symbols.frog,symbols.marsupial,symbols.flying mammal,symbols.game animal,symbols.microbe,symbols.coral,symbols.appetizer,symbols.heritage horse,symbols.jewelry,symbols.bread,symbols.shrub,symbols.pastry,symbols.dish,symbols.handgun,symbols.hymn,symbols.star,symbols.astronomical symbol,symbols.emblem,symbols.heritage livestock,symbols.bat,symbols.oyster,symbols.cocktail,symbols.domestic animal
0,Alabama,AL,"Dec 14, 1819",Heart of Dixie,01,Alabamian,"[{'name': 'Huntsville', 'population': '215006'...",Gulf of Mexico,Cheaha Mountain,http://www.alabama.gov,"$23,606","$49,861",state,"[Camellia State, Yellowhammer State, Cotton St...",Region IV,Ala.,Ala.,7,[CST],[Cfa],Audemus jura nostra defendere,"[Autauga, Baldwin, Barbour, Bibb, Blount, Bull...",Montgomery,32.377716,-86.300568,37,5024279,95,sea level,sea level,500,38,2413,2413,44,736,150,736,52420.07,28,131171,25,30,50645.33,4597,135767,96.61,1774.74,3.39,South,East South Central,Eastern 

In [5]:
# Check data types
df.dtypes

name                          object
postal                        object
date                          object
nickname                      object
fips                          object
                               ...  
symbols.heritage livestock    object
symbols.bat                   object
symbols.oyster                object
symbols.cocktail              object
symbols.domestic animal       object
Length: 203, dtype: object

In [6]:
# Check the counts
df.count()

name                          56
postal                        56
date                          56
nickname                      56
fips                          56
                              ..
symbols.heritage livestock     1
symbols.bat                    1
symbols.oyster                 1
symbols.cocktail               1
symbols.domestic animal        1
Length: 203, dtype: int64

In [16]:
# Slice the data frame to check for nulls
df.iloc[:,196:203].head()

,symbols.astronomical symbol,symbols.emblem,symbols.heritage livestock,symbols.bat,symbols.oyster,symbols.cocktail,symbols.domestic animal
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Drop all territories
states_df = df.drop([2,11,36,41,47])
states_df.head(0)

,name,postal,date,nickname,fips,demonym,cities,lowest_point,highest_point,website,per_capita_income,median_household_income,status,other_nicknames,standard_federal_region,ap_abbreviation,gpo_abbreviation,representatives,time_zones,koppen_climate,motto,subdivisions,capital.name,capital.latitude,capital.longitude,population.density_km,population.total,population.density_mi,elevation.min_ft,elevation.min_m,elevation.mean_ft,elevation.max_rank,elevation.max_ft,elevation.span_ft,elevation.mean_rank,elevation.span_m,elevation.mean_m,elevation.max_m,area.total_mi,area.land_rank,area.land_km,area.water_rank,area.total_rank,area.land_mi,area.water_km,area.total_km,area.land_percent,area.water_mi,area.water_percent,census_bureau.region,census_bureau.division,symbols.mascot,symbols.fruit,symbols.tree fruit,symbols.soil,symbols.game bird,symbols.reptile,symbols.amphibian,symbols.nut,symbols.vegetable,symbols.gem,symbols.rock,symbols.saltwater fish,symbols.mammal,symbols.butterfly,symbols.cake,symbols.bird,symbols.insect,symbols.mineral,symbols.song,symbols.spirit,symbols.tree,symbols.fossil,symbols.freshwater fish,symbols.flower,symbols.horse,symbols.wildflower,symbols.shell,symbols.land mammal,symbols.fish,symbols.bolt action rifle,symbols.dog,symbols.marine mammal,symbols.sport,symbols.plant,symbols.metal,symbols.colors,symbols.drink,symbols.firearm,symbols.dinosaur,symbols.neckwear,symbols.floral emblem,symbols.folk dance,symbols.grape,symbols.fruit and vegetable,symbols.musical instrument,symbols.primitive fish,symbols.grain,symbols.cooking pot,symbols.lichen,symbols.grass,symbols.marine reptile,symbols.dance,symbols.tartan,symbols.gold rush ghost town,symbols.silver rush ghost town,symbols.marine fish,symbols.fabric,symbols.pets,symbols.cactus,symbols.animal,symbols.hero,symbols.shellfish,symbols.children's flower,symbols.heroine,symbols.polka,symbols.marine animal,symbols.macroinvertebrate,symbols.wildlife animal,symbols.dessert,symbols.bug,symbols.beverage,symbols.herb,symbols.saltwater mammal,symbols.heritage cattle breed,symbols.pie,symbols.stone,symbols.tortoise,symbols.saltwater reptile,symbols.prepared food,symbols.waltz,symbols.cold water gamefish,symbols.crop,symbols.raptor,symbols.exercise,symbols.artifact,symbols.snackfood,symbols.prairie grass,symbols.slogan,symbols.river,symbols.rifle,symbols.red wine grape,symbols.white wine grape,symbols.flying fossil,symbols.gun,symbols.music,symbols.wild animal game species,symbols.sports car,symbols.crustacean,symbols.meat pie,symbols.berry,symbols.treat,symbols.ballad,symbols.cat,symbols.team sport,symbols.donut,symbols.bean,symbols.cookie,symbols.muffin,symbols.folk song,symbols.game mammal,symbols.bee,symbols.mushroom,symbols.waterfowl,symbols.toy,symbols.tree nut,symbols.fruit tree,symbols.hockey team,symbols.football team,symbols.endangered species,symbols.aquatic animal,symbols.soft drink,symbols.poultry,symbols.saltwater game fish,symbols.freshwater game fish,symbols.spider,symbols.cryptid,symbols.bush,symbols.freshwater trout,symbols.salamander,symbols.frog,symbols.marsupial,symbols.flying mammal,symbols.game animal,symbols.microbe,symbols.coral,symbols.appetizer,symbols.heritage horse,symbols.jewelry,symbols.bread,symbols.shrub,symbols.pastry,symbols.dish,symbols.handgun,symbols.hymn,symbols.star,symbols.astronomical symbol,symbols.emblem,symbols.heritage livestock,symbols.bat,symbols.oyster,symbols.cocktail,symbols.domestic animal


In [18]:
# Drop half of the columns that have nulls
clean_states_df = states_df.drop(columns=['status','other_nicknames','standard_federal_region','status','other_nicknames','standard_federal_region','koppen_climate','population.density_km','elevation.min_ft','elevation.min_m','elevation.mean_ft','elevation.span_ft','elevation.mean_rank','elevation.span_m','elevation.mean_m','elevation.max_m','area.land_km','area.water_rank','area.total_rank','area.land_mi','area.water_km','area.total_km','area.land_percent','area.water_mi','area.water_percent','symbols.mascot','symbols.fruit','symbols.tree fruit','symbols.soil','symbols.game bird','symbols.reptile','symbols.amphibian','symbols.nut','symbols.vegetable','symbols.rock','symbols.saltwater fish','symbols.mammal','symbols.butterfly','symbols.cake','symbols.insect','symbols.mineral','symbols.spirit','symbols.fossil','symbols.freshwater fish','symbols.wildflower','symbols.shell','symbols.land mammal','symbols.fish','symbols.bolt action rifle','symbols.dog','symbols.marine mammal','symbols.sport','symbols.plant','symbols.metal','symbols.colors','symbols.drink','symbols.firearm','symbols.dinosaur','symbols.floral emblem','symbols.folk dance','symbols.grape','symbols.fruit and vegetable','symbols.musical instrument','symbols.primitive fish','symbols.grain','symbols.cooking pot','symbols.lichen','symbols.grass','symbols.marine reptile','symbols.dance','symbols.tartan','symbols.gold rush ghost town', 'symbols.marine animal','symbols.macroinvertebrate','symbols.wildlife animal','symbols.dessert','symbols.bug','symbols.beverage','symbols.herb','symbols.saltwater mammal','symbols.heritage cattle breed','symbols.pie','symbols.stone','symbols.tortoise','symbols.saltwater reptile','symbols.prepared food','symbols.waltz','symbols.cold water gamefish','symbols.crop','symbols.raptor','symbols.exercise','symbols.artifact','symbols.snackfood','symbols.prairie grass','symbols.slogan','symbols.river','symbols.rifle','symbols.red wine grape','symbols.white wine grape','symbols.flying fossil','symbols.gun', 'symbols.wild animal game species',	'symbols.sports car', 'symbols.crustacean',	'symbols.meat pie','symbols.berry',	'symbols.treat','symbols.ballad','symbols.cat','symbols.team sport','symbols.donut','symbols.bean',	'symbols.cookie','symbols.muffin','symbols.folk song', 'symbols.bee','symbols.mushroom', 'symbols.waterfowl','symbols.toy','symbols.tree nut','symbols.fruit tree','symbols.hockey team','symbols.football team','symbols.endangered species','symbols.soft drink','symbols.poultry','symbols.saltwater game fish','symbols.freshwater game fish','symbols.spider','symbols.cryptid','symbols.bush','symbols.freshwater trout','symbols.salamander', 'symbols.marsupial','symbols.flying mammal','symbols.game animal','symbols.microbe','symbols.coral','symbols.appetizer','symbols.heritage horse','symbols.jewelry','symbols.bread','symbols.shrub','symbols.pastry','symbols.dish','symbols.handgun','symbols.hymn', 'symbols.astronomical symbol','symbols.emblem','symbols.heritage livestock','symbols.bat','symbols.oyster','symbols.cocktail','symbols.domestic animal','symbols.hero','symbols.shellfish',"symbols.children's flower",'symbols.heroine','symbols.polka','symbols.music','symbols.game mammal','symbols.aquatic animal','symbols.frog','symbols.star','symbols.horse','symbols.neckwear','symbols.silver rush ghost town','symbols.marine fish','symbols.fabric',	'symbols.pets',	'symbols.cactus','symbols.animal','population.density_mi','ap_abbreviation','gpo_abbreviation','symbols.flower','symbols.song','symbols.gem','subdivisions','census_bureau.region','census_bureau.division','symbols.bird'])
clean_states_df.head()

,name,postal,date,nickname,fips,demonym,cities,lowest_point,highest_point,website,per_capita_income,median_household_income,representatives,time_zones,motto,capital.name,capital.latitude,capital.longitude,population.total,elevation.max_rank,elevation.max_ft,area.total_mi,area.land_rank,symbols.tree
0,Alabama,AL,"Dec 14, 1819",Heart of Dixie,01,Alabamian,"[{'name': 'Huntsville', 'population': '215006'...",Gulf of Mexico,Cheaha Mountain,http://www.alabama.gov,"$23,606","$49,861",7,[CST],Audemus jura nostra defendere,Montgomery,32.377716,-86.300568,5024279,38,2413,52420.07,28,Southern Longleaf Pine
1,Alaska,AK,"Jan 3, 1959",The Last Frontier,02,Alaskan,"[{'name': 'Anchorage', 'population': '291538'}...","Gulf of Alaska, Bering Sea, and Arctic Ocean",Denali,http://alaska.gov/,"$33,062","$74,346",1,"[AKST, HST]",North to the Future,Juneau,58.301598,-134.420212,733391,1,20310,665384.04,1,Sitka Spruce
3,Arizona,AZ,"Feb 14, 1912",Grand Canyon State,04,Arizonan,"[{'name': 'Phoenix', 'population': '1660272'},...","Colorado River at Sonora border, San Luis, Ari...",Humphreys Peak,https://az.gov/,"$25,715","$59,246",9,[MST],Ditat Deus,Phoenix,33.448143,-112.096962,7151502,12,12637,113990.3,6,Palo Verde
4,Arkansas,AR,"Jun 15, 1836",Natural State,05,Arkansan,"[{'name': 'Little Rock', 'population': '197881...",Ouachita River at Louisiana border,Mount Magazine,http://www.arkansas.gov/,"$22,883","$47,062",4,[CST],Regnat populus,Little Rock,34.746613,-92.288986,3011524,37,2753,53178.55,27,Pine Tree
5,California,CA,"Sep 9, 1850",Golden State,06,Californian,"[{'name': 'Los Angeles', 'population': '399798...",Badwater Basin in Death Valley,Mount Whitney,https://www.ca.gov/,"$30,441","$75,277",53,[PST],Eureka,Sacramento,38.576668,-121.493629,39538223,2,14505,163694.74,3,"Coast Redwood,Giant Sequoia"


In [20]:
# Show all columns of data frame
pd.options.display.max_columns = None
clean_states_df = clean_states_df.replace({np.nan: None})
clean_states_df.to_html('states.html',index=False,classes=["table","table-striped","table-bordered"])
clean_states_df.head()

,name,postal,date,nickname,fips,demonym,cities,lowest_point,highest_point,website,per_capita_income,median_household_income,representatives,time_zones,motto,capital.name,capital.latitude,capital.longitude,population.total,elevation.max_rank,elevation.max_ft,area.total_mi,area.land_rank,symbols.tree
0,Alabama,AL,"Dec 14, 1819",Heart of Dixie,01,Alabamian,"[{'name': 'Huntsville', 'population': '215006'...",Gulf of Mexico,Cheaha Mountain,http://www.alabama.gov,"$23,606","$49,861",7,[CST],Audemus jura nostra defendere,Montgomery,32.377716,-86.300568,5024279,38,2413,52420.07,28,Southern Longleaf Pine
1,Alaska,AK,"Jan 3, 1959",The Last Frontier,02,Alaskan,"[{'name': 'Anchorage', 'population': '291538'}...","Gulf of Alaska, Bering Sea, and Arctic Ocean",Denali,http://alaska.gov/,"$33,062","$74,346",1,"[AKST, HST]",North to the Future,Juneau,58.301598,-134.420212,733391,1,20310,665384.04,1,Sitka Spruce
3,Arizona,AZ,"Feb 14, 1912",Grand Canyon State,04,Arizonan,"[{'name': 'Phoenix', 'population': '1660272'},...","Colorado River at Sonora border, San Luis, Ari...",Humphreys Peak,https://az.gov/,"$25,715","$59,246",9,[MST],Ditat Deus,Phoenix,33.448143,-112.096962,7151502,12,12637,113990.3,6,Palo Verde
4,Arkansas,AR,"Jun 15, 1836",Natural State,05,Arkansan,"[{'name': 'Little Rock', 'population': '197881...",Ouachita River at Louisiana border,Mount Magazine,http://www.arkansas.gov/,"$22,883","$47,062",4,[CST],Regnat populus,Little Rock,34.746613,-92.288986,3011524,37,2753,53178.55,27,Pine Tree
5,California,CA,"Sep 9, 1850",Golden State,06,Californian,"[{'name': 'Los Angeles', 'population': '399798...",Badwater Basin in Death Valley,Mount Whitney,https://www.ca.gov/,"$30,441","$75,277",53,[PST],Eureka,Sacramento,38.576668,-121.493629,39538223,2,14505,163694.74,3,"Coast Redwood,Giant Sequoia"


In [22]:
# Save url as variable
url = "https://jsearch.p.rapidapi.com/search"

# Set querystring 
querystring = {"query":"Entry Level Data Scientist in USA","num_pages":"20"}

# Set headers
headers = {
	"X-RapidAPI-Key": "575ca32490msh5a85e1221a9daf0p10835bjsnfdf17e56a208",
	"X-RapidAPI-Host": "jsearch.p.rapidapi.com"
}

# Make a GET request and save in variable
response = requests.request("GET", url, headers=headers, params=querystring)
entry_level_data_scientists = response.json()

# Write the data to a json file
with open('entry_level_data_scientists.json', 'w') as outfile:
    json.dump(entry_level_data_scientists, outfile, indent=4)


# Print response
#rint(response.text).head()

In [23]:
# Save url as variable
url = "https://jsearch.p.rapidapi.com/search"

# Set querystring
querystring = {"query":"Entry Level Data Analyst in USA","num_pages":"20"}

# Set headers
headers = {
	"X-RapidAPI-Key": "575ca32490msh5a85e1221a9daf0p10835bjsnfdf17e56a208",
	"X-RapidAPI-Host": "jsearch.p.rapidapi.com"
}

# Make a GET request and save in variable
response = requests.request("GET", url, headers=headers, params=querystring)
entry_level_data_analyst = response.json()

# Write the data to a json file
with open('entry_level_data_analyst.json', 'w') as outfile:
    json.dump(entry_level_data_analyst, outfile, indent=4)


# Print response
#print(response.text)

In [24]:
# Select certain columns to be used and turn json file into a pandas data frame 
columns = ["employer_name","job_title","job_apply_link","job_description","job_is_remote","job_city","job_state","job_country","job_latitude","job_longitude"]
df = pd.json_normalize(entry_level_data_analyst, 'data')
new_df = df.loc[:, columns]
new_df.head()

,employer_name,job_title,job_apply_link,job_description,job_is_remote,job_city,job_state,job_country,job_latitude,job_longitude
0,"Staffigo Technical Services, LLC",Entry Level Data Analyst,https://www.learn4good.com/jobs/las-vegas/neva...,About us\n\nStaffigo is in the business of pro...,False,Las Vegas,NV,US,36.171562,-115.13910
1,SynergisticIT,Entry Level Data Analyst,https://www.linkedin.com/jobs/view/entry-level...,About Us\n\nSynergistic IT is a full-service s...,False,Normal,IL,US,40.514202,-88.99063
2,"Staffigo Technical Services, LLC",Entry Level Data Analyst,https://www.adzuna.com/details/3881207174,Job description\n\nAbout us\n\nStaffigo is in ...,False,Raleigh,MS,US,32.033490,-89.52229
3,"Staffigo Technical Services, LLC",Entry Level Data Analyst,https://www.ziprecruiter.com/c/Staffigo-Techni...,About us\n\nStaffigo is in the business of pro...,False,Roswell,GA,US,34.023243,-84.36156
4,TRESUME,Entry Level Data Analyst,https://www.glassdoor.com/job-listing/entry-le...,We are seeking an Entry level Data Analyst to ...,False,Reston,VA,US,38.958630,-77.35700


In [25]:
# Select certain columns to be used and turn json file into a pandas data frame 
columns = ["employer_name","job_title","job_apply_link","job_description","job_is_remote","job_city","job_state","job_country","job_latitude","job_longitude"]
df2 = pd.json_normalize(entry_level_data_scientists, 'data')
new_data_df = df2.loc[:, columns]
new_data_df.head()

,employer_name,job_title,job_apply_link,job_description,job_is_remote,job_city,job_state,job_country,job_latitude,job_longitude
0,National Security Agency,Data Scientist - Entry to Expert Level (Maryla...,https://www.linkedin.com/jobs/view/data-scient...,Responsibilities\n\nData science at the Nation...,False,Fort Meade,MD,US,39.104263,-76.734240
1,Eurofins USA BioPharma Services,Entry Level Data Scientist,https://www.recruit.net/job/entry-level-jobs/E...,Job Description\n\nNOT A CODING OR DATA SCIENC...,False,None,AZ,US,34.048927,-111.093735
2,I28 Technologies,Entry Level Data Scientist-333325,https://www.glassdoor.com/job-listing/entry-le...,"• Analysis tools, defining, and implementing i...",False,Marysville,MI,US,42.912530,-82.486860
3,SynergisticIT,Entry Level Data Scientist,https://www.linkedin.com/jobs/view/entry-level...,About Us\n\nSynergistic IT is a full-service s...,False,Lansing,MI,US,42.732536,-84.555534
4,Lawrence Livermore National Laboratory,Data Scientist - Entry Level,https://www.karkidi.com/job-details/29968-data...,We have openings for Data Scientists to provid...,False,Livermore,CA,US,37.681870,-121.768470


In [26]:
# Add the two data frames together 
result_df = pd.concat([new_df, new_data_df], axis=0)
result_df = result_df.replace({np.nan: None})
result_df = result_df.replace({'\n': ' '}, regex=True)
result_df.to_html('jobs.html',index=False,classes=["table","table-striped","table-bordered"])
result_df.head()

,employer_name,job_title,job_apply_link,job_description,job_is_remote,job_city,job_state,job_country,job_latitude,job_longitude
0,"Staffigo Technical Services, LLC",Entry Level Data Analyst,https://www.learn4good.com/jobs/las-vegas/neva...,About us Staffigo is in the business of provi...,False,Las Vegas,NV,US,36.171562,-115.13910
1,SynergisticIT,Entry Level Data Analyst,https://www.linkedin.com/jobs/view/entry-level...,About Us Synergistic IT is a full-service sta...,False,Normal,IL,US,40.514202,-88.99063
2,"Staffigo Technical Services, LLC",Entry Level Data Analyst,https://www.adzuna.com/details/3881207174,Job description About us Staffigo is in the ...,False,Raleigh,MS,US,32.033490,-89.52229
3,"Staffigo Technical Services, LLC",Entry Level Data Analyst,https://www.ziprecruiter.com/c/Staffigo-Techni...,About us Staffigo is in the business of provi...,False,Roswell,GA,US,34.023243,-84.36156
4,TRESUME,Entry Level Data Analyst,https://www.glassdoor.com/job-listing/entry-le...,We are seeking an Entry level Data Analyst to ...,False,Reston,VA,US,38.958630,-77.35700


In [ ]:
# Connect to the MongoDB server
client = MongoClient()

# Choose the database you want to use
db = client.data

# Insert the clean_states_df data frame into a collection named "clean_states"
db.states.insert_many(clean_states_df.to_dict('records'))

# Insert the result_df data frame into a collection named "result"
db.jobs.insert_many(result_df.to_dict('records'))